In [1]:
import time
import warnings
import logging
import tensorflow as tf

### Decorate functions with tf.function

Functions can be faster than eager code, especially for graphs with many small ops. But for graphs with a few expensive ops (like convolutions), you may not see much speedup.

In [2]:
@tf.function
def add(a, b):
    return a + b

@tf.function
def sub(a, b):
    return a - b

@tf.function
def mul(a, b):
    return a * b

@tf.function
def div(a, b):
    return a / b

In [3]:
print(add(tf.constant(5), tf.constant(2)))

tf.Tensor(7, shape=(), dtype=int32)


In [4]:
print(sub(tf.constant(5), tf.constant(2)))

tf.Tensor(3, shape=(), dtype=int32)


In [5]:
print(mul(tf.constant(5), tf.constant(2)))

tf.Tensor(10, shape=(), dtype=int32)


In [6]:
print(div(tf.constant(5), tf.constant(2)))

tf.Tensor(2.5, shape=(), dtype=float64)


### Operate on variables and tensors, invoke nested functions

In [7]:
@tf.function
def matmul(a, b):
    return tf.matmul(a, b)

In [8]:
@tf.function
def linear(m, x, c):
    return add(matmul(m, x), c)

In [9]:
m = tf.constant([[4.0, 5.0, 6.0]], tf.float32)

m

<tf.Tensor: shape=(1, 3), dtype=float32, numpy=array([[4., 5., 6.]], dtype=float32)>

In [10]:
x = tf.Variable([[100.0], [100.0], [100.0]], tf.float32)

x

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[100.],
       [100.],
       [100.]], dtype=float32)>

In [11]:
c = tf.constant([[1.0]], tf.float32)

c

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1.]], dtype=float32)>

In [12]:
linear(m, x, c)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[1501.]], dtype=float32)>

### Convert regular Python code to TensorFlow constructs

To help users avoid having to rewrite their code when adding @tf.function, AutoGraph converts a subset of Python constructs into their TensorFlow equivalents.

May use data-dependent control flow, including if, for, while break, continue and return statements

In [13]:
@tf.function
def pos_neg_check(x):
    reduce_sum = tf.reduce_sum(x)

    if reduce_sum > 0:
        return tf.constant(1)

    elif reduce_sum == 0:
        return tf.constant(0)
    
    else:
        return tf.constant(-1)

In [14]:
pos_neg_check(tf.constant([100, 100]))

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [15]:
pos_neg_check(tf.constant([100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=0>

In [16]:
pos_neg_check(tf.constant([-100, -100]))

<tf.Tensor: shape=(), dtype=int32, numpy=-1>

### Operations with side effects

May also use ops with side effects, such as tf.print, tf.Variable and others.

In [17]:
num = tf.Variable(7)

In [18]:
@tf.function
def add_times(x):
    for i in tf.range(x):
        num.assign_add(x)

In [19]:
add_times(5)

In [20]:
print(num)

<tf.Variable 'Variable:0' shape=() dtype=int32, numpy=32>


### In-order code execution

Dependencies in the code are automatically resolved based on the order in which the code is written

In [21]:
a = tf.Variable(1.0)

b = tf.Variable(2.0)

In [22]:
@tf.function
def f(x, y):
    
    a.assign(y * b)
    
    b.assign_add(x * a)
    
    return a + b

In [23]:
f(1, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

### Polymorphism and tracing

Python's dynamic typing means that you can call functions with a variety of argument types, and Python will do something different in each scenario.

On the other hand, TensorFlow graphs require static dtypes and shape dimensions. tf.function bridges this gap by retracing the function when necessary to generate the correct graphs. Most of the subtlety of tf.function usage stems from this retracing behavior.

In [24]:
@tf.function
def square(a):
    print("Input a: ", a)
    return a * a

Trace a new graph with floating point inputs

In [25]:
x = tf.Variable([[2, 2], [2, 2]], dtype = tf.float32)

square(x)

Input a:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32>


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4., 4.],
       [4., 4.]], dtype=float32)>

Re-trace the graph, now the inputs are of type integer

In [26]:
y = tf.Variable([[2, 2], [2, 2]], dtype = tf.int32)

square(y)

Input a:  <tf.Variable 'Variable:0' shape=(2, 2) dtype=int32>


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

This time the graph for floating point inputs is not traced, it is simply executed. This means that the print() statement is not executed. Since that is a Python side-effect. Python side-effects are executed only when the graph is traced

In [27]:
z = tf.Variable([[3, 3], [3, 3]], dtype = tf.float32)

square(z)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[9., 9.],
       [9., 9.]], dtype=float32)>

### Use get_concrete_function() to get a concrete trace for a particular type of function

In [28]:
concrete_int_square_fn = square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.int32))

concrete_int_square_fn

Input a:  Tensor("a:0", dtype=int32)


<ConcreteFunction square(a) at 0x2394D21EAC8>

In [29]:
concrete_float_square_fn = square.get_concrete_function(tf.TensorSpec(shape=None, dtype=tf.float32))

concrete_float_square_fn

Input a:  Tensor("a:0", dtype=float32)


<ConcreteFunction square(a) at 0x2394D1E1548>

In [30]:
concrete_int_square_fn(tf.constant([[2, 2], [2, 2]], dtype = tf.int32))

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[4, 4],
       [4, 4]])>

In [31]:
concrete_float_square_fn(tf.constant([[2.1, 2.1], [2.1, 2.1]], dtype = tf.float32))

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[4.4099994, 4.4099994],
       [4.4099994, 4.4099994]], dtype=float32)>

In [32]:
concrete_float_square_fn(tf.constant([[2, 2], [2, 2]], dtype = tf.int32))

InvalidArgumentError: cannot compute __inference_square_240 as input #0(zero-based) was expected to be a float tensor but is a int32 tensor [Op:__inference_square_240]

### Python side effects only happen during tracing

In general, Python side effects (like printing or mutating objects) only happen during tracing. 

In [ ]:
@tf.function
def f(x):
    print("Python execution: ", x)
    tf.print("Graph execution: ", x)

In [ ]:
f(1)

Python execution:  1
Graph execution:  1


In [ ]:
f(1)

Graph execution:  1


In [ ]:
f("Hello tf.function!")

Python execution:  Hello tf.function!
Graph execution:  Hello tf.function!


Appending to Python lists is also a Python side-effect

In [ ]:
arr = []

@tf.function
def f(x):
    for i in range(len(x)):
        arr.append(x[i])    

In [ ]:
f(tf.constant([10, 20, 30]))

In [ ]:
arr

[<tf.Tensor 'strided_slice:0' shape=() dtype=int32>,
 <tf.Tensor 'strided_slice_1:0' shape=() dtype=int32>,
 <tf.Tensor 'strided_slice_2:0' shape=() dtype=int32>]

In [ ]:
@tf.function
def f(x):
    tensor_arr = tf.TensorArray(dtype = tf.int32, size = 0, dynamic_size = True)
    
    for i in range(len(x)):
        tensor_arr = tensor_arr.write(i, x[i])
        
    return tensor_arr.stack()

In [ ]:
result_arr = f(tf.constant([10, 20, 30]))

result_arr

<tf.Tensor: shape=(3,), dtype=int32, numpy=array([10, 20, 30], dtype=int32)>

### Use the tf.py_function() exit hatch to execute side effects

In [ ]:
external_list = []

def side_effect(x):
    print('Python side effect')
    external_list.append(x)

@tf.function
def fn_with_side_effects(x):
    tf.py_function(side_effect, inp=[x], Tout=[])

In [ ]:
fn_with_side_effects(1)

Python side effect


In [ ]:
fn_with_side_effects(2)

Python side effect


In [ ]:
external_list

[<tf.Tensor: shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: shape=(), dtype=int32, numpy=2>]

### Control flow works

for/while --> tf.while_loop (break and continue are supported)

In [ ]:
@tf.function

def some_tanh_fn(x):
    while tf.reduce_sum(x) > 1:
        x = tf.tanh(x)
    
    return x

In [ ]:
some_tanh_fn(tf.random.uniform([10]))

<tf.Tensor: shape=(10,), dtype=float32, numpy=
array([0.10435402, 0.09719177, 0.10527517, 0.10574021, 0.10505985,
       0.10015721, 0.09069003, 0.09638034, 0.09473684, 0.10035308],
      dtype=float32)>

#### Converting a function in eager mode to its Graph representation

Converting a function that works in eager mode to its Graph representation requires to think about the Graph even though we are working in eager mode

In [ ]:
def fn_with_variable_init_eager():

    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.)
    
    y = tf.matmul(a, x) + b

    tf.print("tf_print: ", y)
    
    return y

In [ ]:
fn_with_variable_init_eager()

tf_print:  [[22 22]
 [23 13]]


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

In [ ]:
@tf.function
def fn_with_variable_init_autograph():

    a = tf.constant([[10,10],[11.,1.]])
    x = tf.constant([[1.,0.],[0.,1.]])
    b = tf.Variable(12.)
    
    y = tf.matmul(a, x) + b

    tf.print("tf_print: ", y)
    
    return y

In [ ]:
fn_with_variable_init_autograph()

In [ ]:
class F():
    def __init__(self):
        self._b = None

    @tf.function
    def __call__(self):
        a = tf.constant([[10, 10], [11., 1.]])
        x = tf.constant([[1., 0.], [0., 1.]])
        
        if self._b is None:
            self._b = tf.Variable(12.)
            
        y = tf.matmul(a, x) + self._b
        print(y)

        tf.print("tf_print: ", y)
        return y

fn_with_variable_init_autograph = F()
fn_with_variable_init_autograph()

Tensor("add:0", shape=(2, 2), dtype=float32)
Tensor("add:0", shape=(2, 2), dtype=float32)
tf_print:  [[22 22]
 [23 13]]


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[22., 22.],
       [23., 13.]], dtype=float32)>

In [ ]:
def f(x):
    if x > 0:
        x *= x
    return x
    
print(tf.autograph.to_code(f))    

def tf__f(x):
    do_return = False
    retval_ = ag__.UndefinedReturnValue()
    with ag__.FunctionScope('f', 'fscope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as fscope:

        def get_state():
            return ()

        def set_state(loop_vars):
            pass

        def if_true():
            (x_1,) = (x,)
            x_1 *= x_1
            return x_1

        def if_false():
            return x
        cond = (x > 0)
        x = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('x',), ())
        try:
            do_return = True
            retval_ = fscope.mark_return_value(x)
        except:
            do_return = False
            raise
    (do_return,)
    return ag__.retval(retval_)



#### AutoGraph is highly optimized and works well when the input is a tf.Tensor object

In [ ]:
@tf.function
def g(x):
    return x

start = time.time()
for i in tf.range(2000):
    g(i)
end = time.time()

print("tf.Tensor time elapsed: ", (end-start))

tf.Tensor time elapsed:  0.5737230777740479


In [ ]:
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').disabled = True

In [ ]:
start = time.time()
for i in range(2000):
    g(i)
end = time.time()

print("Native type time elapsed: ", (end-start))

Native type time elapsed:  12.941787004470825
